In [1]:
using DataStructures
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## funzione hollowBall()

In [2]:
function hollowBall(r=1., R=1., angle1=pi, angle2=2*pi)
    function hollowBall0(shape=[24, 36, 3])
        V, CV = cuboidGrid(shape)
        V = [angle1/shape[1] 0 0; 0 angle2/shape[2] 0; 0 0 (R-r)/shape[3]]*V
        V = broadcast(+, V, [-(angle1)/2, -(angle2)/2, r])
        W = [V[:, k] for k=1:size(V, 2)]
        V = hcat(map(p->let
        (u, v, w)=p; [w*cos(u)*cos(v); w*cos(u)*sin(v); w*sin(u)] end, W)...)
        W, CW = simplifyCells(V, CV)
        return W, CW
    end
    return hollowBall0
end


hollowBall (generic function with 5 methods)

In [19]:
@code_warntype Lar.hollowBall()();

Variables
  #self#::LinearAlgebraicRepresentation.var"#hollowBall0#323"{Float64,Float64,Irrational{:π},Float64}

Body::Tuple{Array,Array{Array{Int64,1},1}}
1 ─ %1 = Base.vect(24, 36, 3)::Array{Int64,1}
│   %2 = (#self#)(%1)::Tuple{Array,Array{Array{Int64,1},1}}
└──      return %2


In [4]:
@btime W,CW = Lar.hollowBall()();

  19.464 ms (260857 allocations: 21.48 MiB)


In [5]:
@btime V, CV = Lar.hollowBall(1, 2, pi/2, pi/2)([6, 12, 4]);

  2.277 ms (27526 allocations: 2.44 MiB)


## Uso della macro @threads 

#### Per settare il numero di threads pari a 2 ho dovuto per forza eseguire (come scritto nella guida di julia) i seguenti passaggi :  export JULIA_NUM_THREADS=2  , set JULIA_NUM_THREADS=2 , il numero di threads va settato in base alla macchina detenuta , se non lo si fa l'uso dei threads è nullo .

In [6]:
using Base.Threads
Threads.nthreads() = 2  #setto il numero di threads pari a 2
nthreads()

2

In [16]:
function hollowBall1(r=1., R=1., angle1=pi, angle2=2*pi)
    function hollowBall01(shape=[24, 36, 3])
        V, CV = Lar.cuboidGrid(shape)
        V = [angle1/shape[1] 0 0; 0 angle2/shape[2] 0; 0 0 (R-r)/shape[3]]*V
        V = broadcast(+, V, [-(angle1)/2, -(angle2)/2, r])
        n = size(V,2)
        W = Array{Array{Float64,1}}(undef, n)
        
        @inbounds @threads for k=1:n
         W[k] = V[:,k] 
    end
        V = hcat(map(p->let
        (u, v, w)=p; [w*cos(u)*cos(v); w*cos(u)*sin(v); w*sin(u)] end, W)...)
        W, CW = Lar.simplifyCells(V, CV)
        return W, CW
    end
    return hollowBall01
end


hollowBall1 (generic function with 5 methods)

In [18]:
@btime W,CW = hollowBall1()();

  19.430 ms (264048 allocations: 21.53 MiB)


In [17]:
@btime V, CV = hollowBall1(1, 2, pi/2, pi/2)([6, 12, 4]);

  2.293 ms (27527 allocations: 2.44 MiB)


Sorprendente vedere quando si passano dei parametri come tutti i valori/tempistiche cambino e anche di molto .